## Object Detection Basics

In [1]:
import numpy as np
import cv2

### Finding Waldo Mini-Project

In [ ]:
# using the mathematical correlation algorithm

# load beach image
image_beach = cv2.imread('WaldoBeach.jpg')
cv2.imshow('Where is Waldo?', image_beach)
cv2.waitKey(0)

# load waldo image
image_waldo = cv2.imread('waldo.jpg',0)
# how waldo looks
cv2.imshow('How Waldo Looks?', image_waldo)
cv2.waitKey(0)

gray = cv2.cvtColor(image_beach, cv2.COLOR_BGR2GRAY)
result = cv2.matchTemplate(gray, image_waldo, cv2.TM_CCOEFF)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# creating bounding box around waldo location
top_left = max_loc
bottom_right = (top_left[0]+60, top_left[1]+60)
cv2.rectangle(image_beach, top_left, bottom_right, (0,0,255),3)

cv2.imshow('Where is Waldo?', image_beach)
cv2.waitKey(0)
cv2.destroyAllWindows()
# the above method is though not very efficient with images in rotation, scale, color, etc.

### Corners as Features

In [ ]:
# using the Harris Corner Algorithm

# load image then make it grayscale
image = cv2.imread('chess.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# The cornerHarris function requires the array datatype to be float32
gray = np.float32(gray)

harris_corners = cv2.cornerHarris(gray, 3, 3, 0.05)

#We use dilation of the corner points to enlarge them\
kernel = np.ones((7,7),np.uint8)
harris_corners = cv2.dilate(harris_corners, kernel, iterations = 2)

# Threshold for an optimal value, it may vary depending on the image.
image[harris_corners > 0.025 * harris_corners.max() ] = [255, 127, 127]

cv2.imshow('Harris Corners', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# improved version of the Harris Corner Algorithm called Good Features to Track

image = cv2.imread('chess.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# We specific the top 50 corners
corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10) 
# changing the 10 to 150 will reduce corners

# we are iterating through each corner location and plot rectangle
for corner in corners:
    x, y = corner[0]
    x = int(x)
    y = int(y)
    cv2.rectangle(image,(x-10,y-10),(x+10,y+10),(0,255,0), 2)
    
cv2.imshow("Corners Found", image)
cv2.waitKey()
cv2.destroyAllWindows()

# the corner matching is good for rotations, translations and slight photometric changes
# the problems with corners as features are that they are intolerant
# as the intensity of photometric changes, or scaling occurs, the 

## Face Detection
### Download the Classifier from https://github.com/opencv/opencv/tree/master/data/haarcascades
### Download the file "haarcascade_frontalface_default.xml"

In [13]:
# face detection using haar cascade classifier

img = cv2.imread('family.jpg')
image = img.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# loading the classifier
face_cascade = cv2.CascadeClassifier('./Model/haarcascade_frontalface_default.xml')

# appling gray image to classifier
# detect multiscale2 will return two arguements
box,detections = face_cascade.detectMultiScale2(gray,minNeighbors=8)

# drawing the box around the faces

for x,y,w,h in box:
    
    cv2.rectangle(image, (x,y),(x+w,y+h),(55,255,0),2)

cv2.imshow('face detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Realtime Face Detection

In [2]:
cap = cv2.VideoCapture(0)

# loading the classifier
face_cascade = cv2.CascadeClassifier('./Model/haarcascade_frontalface_default.xml')

# creating the function for detection
def face_detection(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return frame

while True:
    ret, frame = cap.read()
    if ret == False:
        break
        
    img_detect = face_detection(frame)
    cv2.imshow('Real Time Face Detection', img_detect)
    if cv2.waitKey(1) == ord('a'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Single Shot MultiBox Detector (SSD)

In [2]:
img = cv2.imread('family.jpg')

cv2.imshow('Family', img)

# load the model file
face_detection_model = cv2.dnn.readNetFromCaffe('./Model/deploy.prototxt.txt',
                                                './Model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

def face_detection_dnn(img):    
    # step-1: blob from image
    blob = cv2.dnn.blobFromImage(img,1,(300,300),(104, 177, 123), swapRB = True)

    # step-2: blob as input
    face_detection_model.setInput(blob)

    # step-3: blob from image
    detections = face_detection_model.forward()

    # step-4: drawing bounding box on top of face detected
    detections = face_detection_model.forward()
    image = img.copy()
    h,w = image.shape[:2]

    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        print(confidence)
        if confidence > 0.5:
            print(confidence)
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype('int')
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])

            # draw rectangle
            cv2.rectangle(image, pt1, pt2, (0,255,0),1)

            text = 'score : {:.0f} %'.format(confidence*100)
            cv2.putText(image, text, pt1, cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3)
            print(text)
            
    return image

# calling the DNN function
img_detect = face_detection_dnn(img)
cv2.imshow('Face Detection',img_detect)

cv2.waitKey(0)
cv2.destroyAllWindows()

# notice that the model does not recognize children faces

0.9727289
0.9727289
score : 97 %
0.88054407
0.88054407
score : 88 %
0.87623346
0.87623346
score : 88 %
0.244389
0.105969496
0.10409418
0.09980881
0.09719076
0.09696427
0.09621548
0.09572011
0.09566083
0.09505963
0.09375062
0.093513444
0.0927581
0.0914893
0.0911388
0.090530105
0.09051935
0.09045245
0.09040067
0.09037213
0.09009691
0.08982997
0.08925753
0.08881682
0.08859604
0.08850476
0.088421926
0.088340625
0.08792565
0.0879188
0.08779121
0.08761558
0.086888984
0.08659669
0.08599464
0.08561438
0.085596584
0.0851363
0.08459191
0.083928145
0.08359648
0.083594896
0.08344072
0.083418906
0.083256766
0.08291109
0.08275107
0.082578845
0.082390055
0.08235901
0.08230178
0.08228826
0.08221967
0.082174756
0.08214777
0.082119115
0.08208945
0.0820624
0.08147503
0.081228144
0.08112873
0.08097729
0.080766015
0.08069194
0.08010789
0.07978206
0.07971615
0.079460844
0.07912766
0.078982554
0.07895328
0.078917675
0.07880469
0.07879086
0.078363575
0.078344144
0.078315295
0.078229845
0.078145206
0.07802485


### Now Realtime Face Detection

In [3]:
cap = cv2.VideoCapture(0)

# load the model file
face_detection_model = cv2.dnn.readNetFromCaffe('./Model/deploy.prototxt.txt',
                                                './Model/res10_300x300_ssd_iter_140000_fp16.caffemodel')

while True:
    ret, frame = cap.read()
    if ret == False:
        break
        
    img_detect = face_detection_dnn(frame)
    cv2.imshow('Real Time Face Detection', img_detect)
    if cv2.waitKey(1) == ord('a'):
        break
        
cap.release()
cv2.destroyAllWindows()

0.9891083
0.9891083
score : 99 %
0.09626978
0.08999542
0.081211746
0.0810499
0.07937894
0.07799988
0.07796762
0.07781213
0.07779573
0.0763864
0.076060176
0.0756705
0.07521881
0.0743677
0.07327046
0.07298014
0.0727285
0.07263051
0.07217456
0.07175585
0.07167244
0.071369566
0.07086843
0.07078169
0.070621066
0.07052942
0.07043552
0.07041003
0.07040972
0.07019295
0.07012578
0.070081174
0.0700793
0.070070446
0.06996173
0.06981013
0.0696882
0.06965685
0.06956473
0.069536686
0.06949118
0.069461845
0.06919795
0.06875149
0.06872262
0.06856725
0.06854449
0.06843731
0.0682351
0.068121634
0.06809426
0.06808373
0.06794134
0.06792869
0.067868754
0.067797564
0.067621365
0.06754499
0.067427196
0.06741745
0.06728314
0.06726526
0.06725866
0.066865906
0.0667139
0.06610592
0.066058986
0.06603951
0.06601693
0.06599505
0.065897204
0.06585517
0.06579659
0.06566231
0.065660164
0.0656546
0.06550433
0.06544225
0.065396026
0.06539003
0.065369956
0.06527868
0.065136716
0.06511576
0.065104604
0.06510322
0.06500845

0.9164086
0.9164086
score : 92 %
0.09920383
0.0909776
0.08880584
0.08646802
0.08629769
0.08459973
0.08300848
0.08269199
0.08253742
0.08135794
0.08085349
0.078079715
0.07764624
0.077496424
0.076696694
0.07642877
0.07635079
0.07496038
0.07481288
0.07473048
0.074553035
0.07398762
0.073891394
0.07352571
0.07271848
0.07259025
0.0721408
0.071834765
0.071778834
0.071422495
0.07141781
0.07110241
0.07096019
0.07051961
0.07049298
0.070462294
0.07029868
0.070165336
0.07010626
0.069942184
0.06965276
0.06946869
0.06933327
0.06911051
0.06907461
0.06904921
0.06904516
0.06903786
0.06858977
0.06855956
0.06855615
0.06847243
0.06824335
0.06814323
0.06797045
0.06785437
0.06779799
0.0677455
0.06766578
0.06746786
0.06734398
0.067232855
0.06715227
0.067094326
0.06707541
0.06704342
0.0667712
0.066621624
0.06654712
0.06637733
0.06628949
0.066288695
0.06620629
0.0662022
0.06614284
0.066125765
0.06589737
0.065893106
0.06584664
0.06584185
0.06567099
0.065664284
0.065599665
0.065567434
0.06550634
0.06548664
0.0654

0.97790384
0.97790384
score : 98 %
0.09513314
0.091067
0.09037772
0.07905153
0.07903821
0.078443125
0.077468015
0.07699131
0.07485664
0.07462191
0.07379649
0.072650746
0.07255058
0.072369285
0.07220718
0.071965896
0.07177521
0.071749024
0.070722915
0.07025588
0.07025001
0.06996102
0.06904182
0.06897571
0.06865969
0.06855584
0.06830869
0.06824789
0.0680494
0.06797381
0.06787105
0.067802906
0.06751032
0.067508236
0.067355834
0.067004666
0.066915765
0.0667845
0.06672832
0.066621505
0.066356644
0.066296935
0.06625631
0.065947145
0.06568622
0.06566149
0.065567575
0.06548833
0.06548761
0.06506007
0.06499224
0.064931035
0.064919144
0.06486125
0.06471791
0.06470336
0.06466292
0.06465174
0.06448271
0.06436883
0.06427787
0.0642564
0.064166546
0.06416283
0.064129375
0.064084485
0.06393965
0.0639289
0.06392031
0.06391963
0.06386405
0.06385045
0.06372526
0.06366715
0.06355878
0.06353798
0.06350082
0.06349242
0.06347079
0.06325654
0.06319099
0.06310699
0.063046366
0.06301616
0.06292591
0.06290029
0.

0.96311796
0.96311796
score : 96 %
0.090918705
0.08938868
0.07813342
0.07662051
0.07588778
0.07557314
0.07513323
0.07351631
0.07340516
0.07293388
0.07225673
0.07210318
0.07157231
0.070789695
0.07059382
0.07002906
0.06981072
0.06919622
0.068989746
0.06879178
0.06854112
0.0682257
0.0680323
0.06794463
0.067899406
0.06784891
0.06781241
0.06760472
0.06750058
0.06748149
0.067075916
0.0669309
0.0665269
0.06639216
0.06607341
0.06568674
0.06557574
0.06553361
0.065490566
0.065349355
0.06522978
0.06517294
0.064982966
0.06491963
0.06491921
0.06486039
0.0647086
0.06456578
0.06453452
0.064464256
0.064304575
0.064102404
0.06392811
0.06381266
0.0636914
0.063624166
0.06351345
0.06350448
0.06343733
0.06329799
0.06327933
0.0631891
0.0629788
0.06296179
0.062959805
0.062918775
0.06291603
0.06291186
0.0629016
0.062850885
0.06284962
0.062831536
0.062794454
0.06275592
0.06263798
0.06260968
0.06259022
0.06254243
0.06250483
0.06250187
0.062428623
0.06240334
0.062336218
0.062322676
0.062280048
0.06226208
0.06224

0.9104493
0.9104493
score : 91 %
0.114464484
0.08309055
0.08055403
0.07643583
0.07625371
0.07517582
0.07510079
0.074214295
0.07397158
0.073599815
0.073395744
0.0721311
0.07190143
0.071207866
0.070664674
0.07040689
0.07040159
0.0703274
0.069898
0.06973944
0.069502726
0.06948325
0.069111414
0.06789986
0.067780875
0.06761264
0.06758971
0.06758854
0.0673364
0.067306556
0.067085415
0.06699701
0.06693563
0.06678693
0.066767976
0.066753425
0.066429675
0.0664086
0.06606784
0.06604966
0.06592707
0.06583902
0.06582097
0.065748505
0.06572703
0.06566469
0.06559564
0.06549884
0.06545303
0.06543639
0.06530679
0.06529069
0.06502719
0.06493673
0.06491969
0.064830646
0.0648244
0.064573854
0.064377464
0.06436027
0.06435183
0.06427309
0.064093426
0.06406514
0.063971944
0.063936405
0.063844405
0.06380673
0.063658156
0.06357524
0.063524425
0.063521124
0.06345664
0.06339664
0.06331496
0.06320745
0.063205905
0.0630937
0.06309111
0.06306468
0.06301195
0.06291716
0.06287836
0.06286608
0.06281299
0.06279304
0.0

0.9987929
0.9987929
score : 100 %
0.098020956
0.08401887
0.079617634
0.07854223
0.07746653
0.07442002
0.07422972
0.07264312
0.07243449
0.07243167
0.072151266
0.07161272
0.07146624
0.07107376
0.06978675
0.06968192
0.069499515
0.069022045
0.068774976
0.068688825
0.06839071
0.06814887
0.06812858
0.068073474
0.067991145
0.06796508
0.067811355
0.0676603
0.06756294
0.067559965
0.0673272
0.06722489
0.06715447
0.06704078
0.06700267
0.06678858
0.066760406
0.066740304
0.06666317
0.06658371
0.06651879
0.06651669
0.06636542
0.06616423
0.065985255
0.065865494
0.0656915
0.06536354
0.06531498
0.06511871
0.065024234
0.06492064
0.064749025
0.064690836
0.06466404
0.06460339
0.06448747
0.064424254
0.06441185
0.06437873
0.06433251
0.064297274
0.064137004
0.06403976
0.06399911
0.06399813
0.06390815
0.0638742
0.06386337
0.063733496
0.06361455
0.06357019
0.063404545
0.06336706
0.06325909
0.06320372
0.06319751
0.06309875
0.06298757
0.06298047
0.06295419
0.06288847
0.06288583
0.062878795
0.06286469
0.062801525